In [1]:
import json
import numpy as np
from cassandra.cluster import Cluster
from utils import dumpNeighborhood, Vecindario, Restaurante

# Falta saber si son poligonos en un plano o puntos en un globo

## Transformación de datos restaurantes

In [2]:
with open("./restaurants.json", "r") as file:
    with open('./restaurants.csv', 'w') as fileDump:
        fileDump.write('id,location,name,type\n')
        while True:
            line = file.readline()
            if line == '':
                break
            restaurant = json.loads(line)
            restaurant['location']['coordinates'].reverse()
            dumpLine = "{0},\"{1}\",\"{2}\",\"{3}\"\n".format(restaurant["_id"]["$oid"],
                                                   restaurant['location']['coordinates'],
                                                   restaurant["name"],
                                                   restaurant["location"]["type"])
            fileDump.write(dumpLine)

### Transformación de vecinos

In [3]:
with open("./neighborhoods.json", "r") as file:
    with open("./neighborhoods.csv", "w") as dumpFile:
        dumpFile.write(
            'id, "polygonNumber", "pointNumber",name, type, point\n')
        while True:
            line = file.readline()
            if line == '':
                break
            neighborhood = json.loads(line)
            dumpNeighborhood(neighborhood, dumpFile)

## Chequear estructura de Polygon y MultiPolygon

In [4]:
with open("./neighborhoods.json", "r") as file:
    type1 = ''
    while type1 != 'MultiPolygon':
        obj = json.loads(file.readline())
        type1 = obj["geometry"]["type"]

In [5]:
Vecindario(obj["_id"]["$oid"], obj['geometry']
           ['type'], obj['geometry']['coordinates'])

TypeError: float() argument must be a string or a number, not 'list_reverseiterator'

## Generar información para la consulta restaurantes por vecindario

In [ ]:
with open("./neighborhoods.json", "r") as file:
    i = 0
    neighborhoods = []
    while True:
        line = file.readline()
        if line == '':
            break
        obj = json.loads(line)
        neighborhoods.append(Vecindario(
            obj["_id"]["$oid"], obj['geometry']['type'], obj['geometry']['coordinates']))

In [ ]:
with open("./restaurants.json", "r") as file:
    restaurants = []
    while True:
        line = file.readline()
        if line == '':
            break
        obj = json.loads(line)
        restaurants.append(Restaurante(
            obj['_id']['$oid'], obj['location']['coordinates']))

In [ ]:
tuplas = []
for restaurant in restaurants:
    for neighborhood in neighborhoods:
        estaDentro = neighborhood.estaDentro(restaurant)
        if estaDentro is not None:
            tuplas.append(estaDentro)
            break

In [ ]:
if len(restaurants) - len(tuplas) == 0:
    print('Todos los restaurantes están dentro de un vecindario')
else:
    print('Hay restaurantes que no se encuentran en ningún vecindario')

In [ ]:
with open('RestaurantesPorVecindario.csv', 'w') as file:
    file.write('id_vecindario,polygonNumber,id_restaurante\n')
    for tupla in tuplas:
        file.write(f"'{tupla[0]}',{tupla[1]},'{tupla[2]}'\n")